In [ ]:
%matplotlib inline
import math
import string
import random
import random
import bitarray
import mmh3
import matplotlib.pyplot as plt

Probabilistic data structures let us save RAM by sacrificing accuracy. Different types of probabilistic data structures do this differently, and therefore they are more useful for different applications. 

## Morris counter

A Morris counter is useful when we need to count large numbers but precission is not that important. This is how it works:

- set the counter at i = 0
- when querying the counter, return 2 ^ i, and increment i with probability 1 / 2 ^ i

So, for instance, the first time we query the counter we obtain the value 1. Next time we will get the value 2. Then we will get the value 2 or 4 with 0.5 probability each, etc. The more we count, the more inaccurate the value is. Therefore, this method is useful when we are interested to know orders of magniture.

If we only use one byte to store the value of i (so that the maximum value of i is 256), we can count up to 2 ^ 256, which would require many more bytes.

In the plot below, the continuous line represents the increment of a normal integer, which requires multiple bytes. The dotted lines represent the increment of three different 1 byte Morris counter. We compare to three runs of the Morris counter due to the random nature of the algorithm. 

In [ ]:
def morris_counter(n):
    it = 0
    i = 0
    while it < n:
        yield 2 ** i
        
        if random.random() <= 1 / (2 ** i):
            i = i + 1
            
        it = it + 1

In [ ]:
fig, ax = plt.subplots()
ax.plot(range(10000), range(10000), lw = 2)
ax.plot(range(10000), list(morris_counter(10000)), '--')
ax.plot(range(10000), list(morris_counter(10000)), '--')
ax.plot(range(10000), list(morris_counter(10000)), '--')
fig.set_figwidth(16)

## Bloom filters

Bloom filters are data structures that were created to answer the quesition of whether we have seen an item before. An item is represented by a set of hash codes. The status of hash codes is stored as a list of booleans. Initially, all values are false. When we insert an item, the index of all its derived hash codes is set to True. To test whether an object was already inserted, we just check the value of its corresponding hash indexes. 

There are no false negatives (if the Bloom filter says that we didn't see an item, we certainly didn't) and there is a 0.5% probability of false positives (if we identify an item as already in the data structure, there is a 0.5% probability that this is due to hash collisions.)

The length of the bool array (the number of bits) and the number of hash values to represent each item depends on the required capacity and error rate. 

In this example we are using bits to represent the bool array instead of booleans, because booleans actually take 4 bits.

In [ ]:
class BloomFilter(object):
    def __init__(self, capacity, error = 0.005):
        '''
        Initialise a Bloom filter with given capacity and a false positive rate
        '''
        self.capacity = capacity
        self.error = error
        self.num_bits = int(-capacity * math.log(error) / math.log(2)**2) + 1
        self.num_hashes = int(self.num_bits * math.log(2) / float(capacity)) + 1
        self.data = bitarray.bitarray(self.num_bits)
        
    def _indexes(self, key):
        h1, h2 = mmh3.hash64(key)
        for i in range(self.num_hashes):
            yield (h1 + i * h2) % self.num_bits
            
    def add(self, key):
        for index in self._indexes(key):
            self.data[index] = True
            
    def __contains__(self, key):
        return all(self.data[index] for index in self._indexes(key))
    
    def __len__(self):
        num_bits_on = self.data.count(True)
        return -1.0 * self.num_bits * \
                      math.log(1.0 - num_bits_on / float(self.num_bits)) / \
                      float(self.num_hashes)
            
    @staticmethod
    def untion(bloom_a, bloom_b):
        assert bloom_a.capacity == bloom_b.capacity
        assert bloom_a.error == bloom_b.error
        
        bloom_union = BloomFilter(bloom_a.capacity, bloom_a.error)
        bloom_union.data = bloom_a.data | bloom_b.data
        
        return bloom_union

Let's create a set of 1000 unique random strings

In [ ]:
words = []
while len(words) < 1000:
    word = ''.join(random.choice(string.ascii_uppercase + string.digits) for _ in range(10))
    if word not in words:
        words.append(word)

And now let's plot the ratio of false positives as we add more words to a BloomFilter, especially as we go beyond its capacity. We will create a BloomFilter with a capacity of 500. We will insert 700 words and use th 300 remaining ones as a test set to calculate the false positives rate. 

In [ ]:
bf = BloomFilter(500)
fp = []
for i in range(700):
    bf.add(words[i])
    falses = 0
    for j in range(700, 1000):
        if words[j] in bf:
            falses = falses + 1
    fp.append(falses/300 * 100)

In [ ]:
fig, ax = plt.subplots()
ax.set_xlabel('inserted words')
ax.set_ylabel('% of false positives (test set)')
ax.axvline(x = 500, color = 'r')
ax.grid(True)
plt.plot(fp)